In [1]:
#If any of these imports don't work, you probably need to do a pip install
import import_ipynb #This is the module that lets you import ipynbs as if they are modules
import numpy as np
import pandas as pd
import re
import nltk 
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import textstat as ts # a module that includes basic text functions
# importing some of my simple functions
from simple_functions_functions_only import phrases, phrase_count, dataframeizer, Ngrams

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jrnoo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


importing Jupyter notebook from simple_functions_functions_only.ipynb


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jrnoo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.decomposition import PCA

In [3]:
def PCAphrases(df,n,numfeatures): # n is the no. of words per phrase, numfeatures is how many components you want
    speechphrases = [phrases(text,n) for text in df["Transcript"]] #This gives us a list of lists of phrases
    allphrases = [] 
    for i in speechphrases: #adds all elements of i to allphrases,
        allphrases += i     #ultimately creating one list of all phrases used in the speeches
    phraselist = np.unique(allphrases) #gets rid of duplicates
    newdf = df.copy()
    for phrase in phraselist: #adds a column for each unique phrase giving the no of times that phrase is used in 
        newdf[phrase] = newdf["Transcript"].apply(lambda text: phrases(text,n).count(phrase)) # each speech
    labels = df["President"]
    features = newdf.drop(["President","Transcript"],axis=1)
    train_vectors,test_vectors,train_labels,test_labels=train_test_split(features,labels)
    pca = PCA(n_components=numfeatures, whiten=True)
    _ = pca.fit(train_vectors)
    newfeatures = pca.transform(features)
    return pd.DataFrame(newfeatures)

In [4]:
def pseudoPCA(df,n,numfeatures): 
    speechphrases = [phrases(text,n) for text in df["Transcript"]]
    allphrases = []
    for i in speechphrases:
        allphrases += i
    phraselist = np.unique(allphrases)
    newdf = df.copy()
    labels = df["President"]
    presidents = np.unique(labels)
    stds = []
    phrasestds = {}
    for phrase in phraselist:
        newdf[phrase] = newdf["Transcript"].apply(lambda text: phrases(text,n).count(phrase))
        aves = []
        for pres in presidents:
            rows = newdf[newdf["President"]==pres] # pick out only the rows corresponding to each president
            aves.append(sum(rows[phrase])/len(rows[phrase])) #avereage number of times each pres uses the phrase
        mean = sum(aves)/len(aves)
        devs = np.array(aves)-mean
        var = sum(devs**2)/len(devs)
        std = var**0.5 # We calculate the standard deviation of the presidents' average no of phrase uses
        stds.append(std)
        phrasestds[phrase] = std
    stds.sort(reverse=True)
    cutoff = stds[n-1] #Only the phrases in the top n std devs will be picked as features 
    for phrase in phraselist:
        if phrasestds[phrase] < cutoff:
            newdf.drop(phrase,axis=1) # get rid of features with std dev too small - they are less significant
    features = newdf.drop(["President","Transcript"],axis=1)
    return features